In [16]:
import glob

import random

save_folder = "data/data_1s_0overlapTEMP/"

files = glob.glob(f'{save_folder}/*')
healthy = []
depressed = []

for file in files:
    id = file.split('/')[-1].split('_')[0]
    if id[0] == "H":
        healthy.append(file)
    else:
        depressed.append(file)

depressed_ids = list(set([file.split('/')[-1].split('_')[0] for file in depressed]))
healthy_ids = list(set([file.split('/')[-1].split('_')[0] for file in healthy]))

# sort
depressed_ids.sort()
healthy_ids.sort()



In [17]:
from tqdm import tqdm

In [18]:
n_folds = 10

difference = 100000
for _ in tqdm(range(1000)):

    # shuffle with random seed
    # random.seed(42)

    random.shuffle(depressed_ids)
    random.shuffle(healthy_ids)
    # randomly sp
    # lit into 10 folds
    depressed_folds = []
    healthy_folds = []

    for i in range(n_folds):
        depressed_folds.append(depressed_ids[i::n_folds])
        healthy_folds.append(healthy_ids[i::n_folds])

    # combine
    folds = []
    for i in range(n_folds):
        folds.append(depressed_folds[i] + healthy_folds[i])

    # with open("data/folds.txt", "w") as f:
    #     f.write("\n".join([",".join(fold) for fold in folds]))

    fold_sizes = []

    for fold in folds:
        fold_files = [file for file in files if file.split('/')[-1].split('_')[0] in fold]
        fold_healthy_files = [file for file in fold_files if file.split('/')[-1][0] == "H"]
        fold_depressed_files = [file for file in fold_files if file.split('/')[-1][0] != "H"]
        fold_sizes.append(len(fold_depressed_files))

    max_fold_difference = max(fold_sizes) - min(fold_sizes)
    if max_fold_difference < difference:
        difference = max_fold_difference
        best_folds = folds

print(difference)


100%|██████████| 1000/1000 [00:54<00:00, 18.43it/s]

116


In [21]:
with open("data/folds.txt") as f:
    lines = f.readlines()

    # remove \n
    lines = [line.replace("\n","") for line in lines]

    folds = [line.split(",") for line in lines]

len([id for fold in folds for id in fold])

58

In [19]:
# save list to file
with open("data/folds.txt", "w") as f:
    f.write("\n".join([",".join(fold) for fold in best_folds]))